In [23]:
import pandas as pd
import requests
import json

In [39]:
ICUBeds = pd.read_csv("ICUsByCounty.csv") 

In [40]:
ICUBeds.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed
0,Alabama,Autauga,6,55036,10523,19.1,1754.0
1,Alabama,Baldwin,51,203360,53519,26.3,1049.0
2,Alabama,Barbour,5,26201,6150,23.5,1230.0
3,Alabama,Bibb,0,22580,4773,21.1,NaN
4,Alabama,Blount,6,57667,13600,23.6,2267.0


In [152]:
api_response = requests.get('https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000')
hospitals = pd.json_normalize(api_response.json()['features'])
df_length = len(hospitals)
api_target = 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000'

while True:
    api_response = requests.get(api_target+'&resultOffset='+str(df_length))
    new = pd.json_normalize(api_response.json()['features'])
    hospitals = hospitals.append(new)
    if (df_length == len(hospitals)):
        break
    df_length = len(hospitals)

In [153]:
len(hospitals)

7581

In [154]:
len(ICUBeds)

3142

In [183]:
ICUBeds.sum()

State                                  AlabamaAlabamaAlabamaAlabamaAlabamaAlabamaAlab...
County                                 AutaugaBaldwinBarbourBibbBlountBullockButlerCa...
ICU Beds                                                                           74787
Total Population                                                               321004407
Population Aged 60+                                                             66956449
Percent of Population Aged 60+                                                   77708.3
Residents Aged 60+ Per Each ICU Bed                                          1.91815e+06
lowercase                              autaugabaldwinbarbourbibbblountbullockbutlerca...
dtype: object

In [155]:
hospitals = hospitals.rename(columns = {'attributes.BEDS': 'Hospital Beds', 'attributes.COUNTY': 'County'})

In [182]:
hospitals.sum()

attributes.OBJECTID                                               28739571
attributes.ID            0005793230005339136200111900230017090028002369...
attributes.NAME          CENTRAL VALLEY GENERAL HOSPITALLOS ROBLES HOSP...
attributes.ADDRESS       1025 NORTH DOUTY STREET150 VIA MERIDA4060 WHIT...
attributes.CITY          HANFORDWESTLAKE VILAGELOS ANGELESHOLLYWOODBALD...
attributes.STATE         CACACACACACACACACACACACACACACACACACAGAGALALALA...
attributes.ZIP           9323091362900239002891706907129101691754905059...
attributes.ZIP4          NOT AVAILABLENOT AVAILABLENOT AVAILABLENOT AVA...
attributes.TELEPHONE     NOT AVAILABLENOT AVAILABLENOT AVAILABLE(323) 4...
attributes.TYPE          GENERAL ACUTE CAREGENERAL ACUTE CAREGENERAL AC...
attributes.STATUS        CLOSEDOPENOPENOPENOPENOPENOPENOPENOPENOPENOPEN...
attributes.POPULATION                                               308361
County                   KINGSVENTURALOS ANGELESLOS ANGELESLOS ANGELESL...
attributes.COUNTYFIPS    

In [196]:
beds = hospitals[['County', 'Hospital Beds']].groupby('County').sum()

In [198]:
beds = beds.assign(lowercase=lambda df: df.index.str.lower())
ICUBeds = ICUBeds.assign(lowercase=lambda df: df.County.str.lower())

In [214]:
result = ICUBeds.merge(beds, how='left', on='lowercase')

In [217]:
result = result.drop('lowercase', axis=1)

In [219]:
result.to_csv('HospitalAndICUBeds.csv')